In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd



base_df = pd.read_csv("/home/samir/Desktop/rudraAnalytics/sub_projects/churn/data/data.csv")
pd.set_option("display.max_columns", None)

df = base_df.copy()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)


def one_hot_encode(df, encode_set=[], dont_encode=[]):
    encoded_df = df.copy()
    for column in encode_set:
        if column in dont_encode:
            continue
        # print(len(df[column].unique()))
        if df[column].dtype == 'object':
            if len(df[column].unique()) == 2:
                # Binary encoding (0 and 1)
                encoded_df[column] = pd.get_dummies(df[column], drop_first=True)
            else:
                # One-hot encoding and using 0 and 1 instead of True and False
                one_hot_encoded = pd.get_dummies(df[column], prefix=column, drop_first=False)
                one_hot_encoded.columns = [f"{column}{i+1}" for i in range(one_hot_encoded.shape[1])]
                encoded_df = pd.concat([encoded_df, one_hot_encoded], axis=1)
                encoded_df.drop(column, axis=1, inplace=True)
                # print(encoded_df)

    return encoded_df

features = [
    'customerID',
    'gender',
    'SeniorCitizen',
    'Partner',
    'Dependents',
    'tenure',
    'PhoneService',
    'MultipleLines',
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'Contract',
    'PaperlessBilling',
    'PaymentMethod',
    'MonthlyCharges',
    'TotalCharges',
    'Churn'
]
dont_label = ['customerID', 'tenure', 'MonthlyCharges', 'TotalCharges']

encoded_df = one_hot_encode(df.drop('customerID', axis=1), features, dont_label)
encoded_df['TotalCharges'].fillna(encoded_df['TotalCharges'].mean(), inplace=True)

# Extract features and target
X = encoded_df.drop('Churn', axis=1)
y = encoded_df['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = xgb.XGBClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Build the final model with the best parameters
final_xgb_model = xgb.XGBClassifier(**best_params)
final_xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = final_xgb_model.predict(X_test)

# Evaluate the model
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Test Accuracy: {accuracy_xgb * 100:.2f}%")
print(f"Best Hyperparameters: {best_params}")


XGBoost Test Accuracy: 81.12%
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


XGBoost Test Accuracy: 81.12%